In [11]:
import t5_encoder
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer
import torch
from accelerate.utils import convert_outputs_to_fp32

from alpaca_farm import data_utils
from argparse import Namespace
import pathlib

import numpy as np
from datasets import load_metric
metric = load_metric('accuracy')

data_args = Namespace()
data_args.prompt_dict_path = pathlib.Path('./prompts/v0_inputs_noinputs.json')
data_args.dataset_path = '../seahorse_data/'
data_args.classification_label_key = 'question4'

training_args = Namespace()
training_args.end_sequence_with_eos = False

In [12]:
def cast_with_native_amp(func, mixed_precision):
    """Almost like how huggingface accelerate cast `model.forward`."""
    if mixed_precision not in ("fp16", "bf16"):
        logger.warning(f"Unknown mixed precision mode: {mixed_precision}, falling back to fp32.")
        return func

    if mixed_precision == "fp16":
        output_func = torch.cuda.amp.autocast(dtype=torch.float16)(func)
    else:
        device_type = "cuda" if torch.cuda.is_available() else "cpu"
        output_func = torch.autocast(device_type=device_type, dtype=torch.bfloat16)(func)
    output_func = convert_outputs_to_fp32(output_func)
    return output_func
    


In [13]:
# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('/mnt/nfs_csail/models/swhan/alpaca_farm/q_four_flant5/')
model.forward = cast_with_native_amp(model.forward, 'bf16')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-large', model_max_length=1024)

In [14]:
from typing import Optional

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    metrics = metric.compute(predictions=predictions, references=labels)
    if np.all(predictions==0) or np.all(predictions==1):
        metrics['pearson'] = 0
    else:
        metrics['pearson'] = np.corrcoef(labels.squeeze(), predictions)[0,1]
    print(metrics)
    return metrics

def format_prompt(example: dict, prompt_dict: dict) -> str:
    """Formats a prompt with a prompt_dict formatter.

    Args:
        example: A dict-like object with required keys "instruction" and "input"
        prompt_dict: Dictionary containing the keys "prompt_noinputs" and "prompt_inputs" which have
            placeholders corresponding to the keys from `example`. E.g. "{instruction}".

    Returns:
        A formatted prompt string.

    Examples
    --------
    >>> format_prompt(dict(instruction="test", input=""), prompt_dict=dict(prompt_noinputs="prompt {instruction} "))
    "prompt test"
    """
    assert "instruction" in example and "input" in example, "Internal error: example missing required keys."

    if example["input"] is None or len(example["input"]) == 0:
        formatted_prompt = prompt_dict["prompt_noinputs"].format_map(example)
    else:
        formatted_prompt = prompt_dict["prompt_inputs"].format_map(example)

    return formatted_prompt


def format_output_word_by_word(example: dict, eos_token: Optional[str] = None, output_key="output") -> str:
    if eos_token is None:
        eos_token = ""
    output = f"{example[output_key]}{eos_token}"
    return output.split()

def format_output(example: dict, eos_token: Optional[str] = None, output_key="output") -> str:
    if eos_token is None:
        eos_token = ""
    output = f"{example[output_key]}{eos_token}"
    return output

def _get_text(example: dict, output_key: str):
    example['instruction'] = INSTRUCTIONS['seahorse_data']
    example['input'] = example['text']
    source = format_prompt(example, prompt_dict=prompt_dict)
    target = format_output(
        example,
        eos_token=tokenizer.eos_token if training_args.end_sequence_with_eos else None,
        output_key=output_key,
    )
    return source + ' ' + target

def _get_text_target_word_by_word(example: dict, output_key: str):
    example['instruction'] = INSTRUCTIONS['seahorse_data']
    example['input'] = example['text']
    source = format_prompt(example, prompt_dict=prompt_dict)
    target = format_output_word_by_word(
        example,
        eos_token=tokenizer.eos_token if training_args.end_sequence_with_eos else None,
        output_key=output_key,
    )
    return [source + ' ' + ' '.join(target[:t]) for t in range(len(target))], target

In [15]:
# data_module = data_utils.make_classification_reward_modeling_data_module(
#         tokenizer=tokenizer,
#         data_args=data_args,
#         training_args=training_args,
#     )
import datasets
from alpaca_farm import utils

prompt_dict = utils.jload(data_args.prompt_dict_path)
data_files = {"train": "train.json", "validation": "validation.json"}
dataset_json = datasets.load_dataset(data_args.dataset_path, data_files=data_files)
dataset_json = dataset_json.filter(lambda example: example['worker_lang'] == 'en-US')
train_dataset = dataset_json['train']
eval_dataset = dataset_json['validation']

In [16]:
import pandas as pd
INSTRUCTIONS = {
    'seahorse_data': "Generate a one-sentence summary of this post.",
}
eval_dict_data = pd.DataFrame(eval_dataset).to_dict(orient="records")

indices_to_remove = []
for i, dict_data in enumerate(eval_dict_data):
    if dict_data['text'] is None:
        indices_to_remove.append(i)
for index in sorted(indices_to_remove, reverse=True):
    del eval_dict_data[index]

print(len(indices_to_remove))

train_dict_data = pd.DataFrame(train_dataset).to_dict(orient="records")

indices_to_remove = []
for i, dict_data in enumerate(train_dict_data):
    if dict_data['text'] is None:
        indices_to_remove.append(i)
for index in sorted(indices_to_remove, reverse=True):
    del train_dict_data[index]

print(len(indices_to_remove))

0
0


In [7]:
eval_dict_data[i]['question4']

IndexError: list index out of range

In [22]:
import torch
from tqdm import trange
num_correct = 0.
num_total = 0.
model = model.cuda()
model.eval()
with torch.no_grad():
    for i in trange(len(train_dataset)):
        sequences = _get_text(train_dict_data[i], 'summary')
        input_ids = tokenizer(sequences, truncation=True, return_tensors='pt')['input_ids']
        # with torch.cuda.amp.autocast(enabled=True, dtype=torch.bfloat16):        
        outputs = model(input_ids.cuda())
        label = train_dict_data[i]['question4']
        pred = torch.argmax(outputs.logits, dim=-1)
        if pred == label:
            num_correct +=1
        num_total +=1

        if i % 100 == 0:
            print(num_correct/num_total)
        rewards = outputs.logits[:, 1] - outputs.logits[:, 0]
        if torch.isnan(rewards).any() or torch.isinf(rewards).any():
            print(rewards)

  0%|                                                                                              | 3/14755 [00:00<17:33, 14.00it/s]

1.0


  1%|▋                                                                                           | 105/14755 [00:03<07:24, 32.93it/s]

0.8118811881188119


  1%|█▎                                                                                          | 208/14755 [00:05<06:28, 37.44it/s]

...

 99%|█████████████████████████████████████████████████████████████████████████████████████████ | 14607/14755 [06:21<00:04, 36.76it/s]

0.8122731319772618


100%|█████████████████████████████████████████████████████████████████████████████████████████▋| 14707/14755 [06:24<00:01, 35.62it/s]

0.8123256921297871


100%|██████████████████████████████████████████████████████████████████████████████████████████| 14755/14755 [06:25<00:00, 38.24it/s]


In [19]:
input_ids

tensor([[ 7255,    19,    46,  8033,    24,  8788,     3,     9,  2491,     6,
             3, 13804,    28,    46,  3785,    24,   795,   856,  2625,     5,
          8733,     3,     9,  1773,    24, 18056,   743,     7,     8,  1690,
             5,  1713, 30345, 21035,    10,  6939,  2206,     3,     9,    80,
            18,  5277,  1433,  9251,    13,    48,   442,     5,  1713, 30345,
            86,  2562,    10,   938, 13197,    71,  3972, 31385, 25688,     6,
          9938,  3529,    37,  7605,    56,   810,     8,  1075, 15328,    11,
         22923, 14219,   577,    16,     3,     9,  2839,  3298,     5,   299,
             8,  8565,    16,  2342,     3,     9,   161,   179,   408,    21,
             8,   628,  6696,    31,     7, 12533,   291,  5009,   598,   165,
           792,  1487,    56,   230,   420,   305,  2394,    51, 10186,    41,
         19853,  2560,    51,   137,  8541,  2315,    13,     8,  1611,  5844,
          7038,    33, 12718,   713,    24,  4030,  

In [ ]:
tokenizer.pad_token_id is None

In [20]:
import torch
from tqdm import trange
num_correct = 0.
num_total = 0.
model = model.cuda()
with torch.no_grad():
    model.eval()
    for i in trange(len(data_module['eval_dataset'])):
        sequences, words = _get_text_target_word_by_word(eval_dict_data[4], 'summary')
        input_ids = tokenizer(sequences, truncation=True, return_tensors='pt')['input_ids']
        outputs = model(input_ids.cuda())
        rewards = outputs.logits[:, 1] - outputs.logits[:, 0]
        outputs = outputs.logits.softmax(dim=-1)
        rewards_normalized = outputs[:, 1] - outputs[:, 0]
        
        x = np.arange(len(rewards_normalized))
        plt.plot(x, rewards_normalized.cpu().numpy())

        for i, word in enumerate(words):
            plt.annotate(word, (x[i], rewards_normalized[i]))

        plt.xlabel('t (time step)')
        plt.ylabel('rewards normalized')
        plt.title('reward model over partial sequences')
        plt.show()
        break

NameError: name 'data_module' is not defined

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(10)
y = np.random.rand(10)
words = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

plt.plot(x, y)

for i, word in enumerate(words):
    plt.annotate(word, (x[i], y[i]))

plt.xlabel('x')
plt.ylabel('y')
plt.title('Line Plot with Labeled Points')
plt.show()